In [ ]:
#Library
library(shiny)
library(shinythemes)
library(ggplot2)
library(dplyr)
library(shinydashboard)

#load data
prod_ranking_model <- read.csv("Product-Rankings.csv", check.names = FALSE)
cust_prod_ranking_model <- read.csv("Customer-Product-Rankings.csv", check.names = FALSE)
cust_correlation_model <- read.csv("Customer-Customer-Correlation-Matrix.csv", row.names = "Party", check.names = FALSE)
prod_correlation_model <- read.csv("Product-Product-Correlation-Matrix.csv", row.names = "Product", check.names = FALSE)

#Load data visualize 
#Column Plot
data1 <- read.csv("Sales-Transactions-Edited.csv")
data1$Date <- as.Date(data1$Date)
data1$Voucher <- as.numeric(data1$Voucher)
data1$Qty <- as.numeric(data1$Qty)
daily_vouchers <- aggregate(Voucher ~ Date, data = data1, sum)

#UI
ui <- shinyUI(dashboardPage(
  dashboardHeader(title = "Product Recommendation", titleWidth = 300),
  dashboardSidebar(
    sidebarMenu(
      menuItem("Home", tabName = "home", icon = icon("home")),
      menuItem("Visualize", tabName = "visualize", icon = icon("chart-bar")),
      menuItem("Recommendation System", tabName = "recommend", icon = icon("thumbs-up"))
    )
  ),
  dashboardBody(
    tabItems(
      tabItem(tabName = "home",
              fluidRow(
                column(width = 6,
                       h3("Information Team:"),
                       textOutput("person1"),
                       textOutput("person2"),
                       textOutput("person3"),
                       textOutput("person4")
                ),
                column(width = 6,
                       h3("Review project:"),
                       HTML("a voluminous roger wool how dearly discourage a tidy band
              \nmover supposing continually command a easy-going 
              \ntool reduction even knottily stake a traumatic 
              \nharass insolence yet optimistically permeate 
              \na wooden realign seed till well coordinate 
              \na foolish castrate wallaby while judgementally 
              \nblemish a adventurous stir jaguar before 
              \npromptly glass a far-flung desensitise swell 
              \nbefore triumphantly prescind a mammoth hesitate snarl.</p>")
                )
              )
      ),
      tabItem(tabName = "visualize",
              fluidRow(
                column(width = 4,
                       selectInput("plotType", "Select Plot Type:",
                                   choices = c("Scatter Plot", "Bar Plot", "Column Plot"),
                                   selected = "Scatter Plot"
                       )
                ),
                column(width = 8,
                       plotOutput("selectedPlot")
                )
              )
      ),
      tabItem(tabName = "recommend",
              fluidRow(
                column(width = 4,
                       tags$h3("Input:"),
                       textInput("txt1", "Enter Customer Name:", ""),
                       actionButton("loginButton", "Login"),
                       textInput("txt2", "Enter Product Name:", ""),
                       actionButton("viewButton", "View")
                ),
                column(width = 8,
                       uiOutput("loggedInUI"),
                       uiOutput("viewedInUI")  
                )
              )
      )
    )
  )
))

  
#Server
server <- function(input, output, session) {
  
  #Output tab Home
  output$person1 <- renderText({
    "Nguyen Ba Hoang Nam - HE161029"
  })
  
  output$person2 <- renderText({
    "   Thai Ba Bao      - HE163102 "
  })
  
  output$person3 <- renderText({
    "   Le Van Huynh     - HE163919"
  })
  
  output$person4 <- renderText({
    "   Do Thanh Dat     - HE161567 "
  })   
  
  #Output tab Visualize  
  #Scatter chart 
  generateScatterPlot <- function() {
    # Code to generate scatter plot goes here
    # Replace with your own data and plot code
    x <- c(1, 2, 3, 4, 5)
    y <- c(10, 5, 12, 9, 7)
    df <- data.frame(x, y)
    ggplot(df, aes(x, y)) +
      geom_point()
  }
  
  # Barplot chart
  # Function to generate bar plot
  generateBarPlot <- function() {
    # Code to generate bar plot goes here
    # Replace with your own data and plot code
    barplot(table(mtcars$cyl))
  }
  generateColumnPlot <- function() {
  # Code to generate bar plot goes here
  # Replace with your own data and plot code
  ggplot(daily_vouchers, aes(x = Date, y = Voucher)) +
  geom_col(fill = "#D55E00")
  }
  
  # Select chart
  output$selectedPlot <- renderPlot({
    selectedPlot <- input$plotType
    
    if (selectedPlot == "Scatter Plot") {
      generateScatterPlot()
    } else if (selectedPlot == "Bar Plot") {
      generateBarPlot()     
    } else if  (selectedPlot == "Column Plot") {
      generateColumnPlot()
        }
  }) 
  
  #Output tab Recommend  
  loginClicked <- reactiveVal(FALSE) # check login
  viewClicked <- reactiveVal(FALSE)  # check view
    
  #Initialize showTabs to FALSE
  showTabs <- FALSE

  observeEvent(input$loginButton, {
    loginClicked(TRUE)
    
    # Validate customer name
    customerName <- input$txt1
    if (!customerName %in% cust_prod_ranking_model$Party) {
      loginClicked(FALSE)
      showModal(modalDialog(
        title = "Error",
        paste("Invalid customer name:", customerName, "Please enter a valid name."),
        footer = tagList(
          actionButton("goBackButton", "Go Back")
        )
      ))
      
      # Set showTabs to false
      showTabs <<- FALSE
      
    } else {
      
      # Set showTabs to true
      showTabs <<- TRUE
      
    }
    
    # Reset viewClicked when logging in again
    viewClicked(FALSE)
  })
  
  observeEvent(input$viewButton, {
    viewClicked(TRUE)
    
    # Validate product name
    productName <- input$txt2
    if (!productName %in% prod_ranking_model$Product) {
      viewClicked(FALSE)
      showModal(modalDialog(
        title = "Error",
        paste("Invalid product name:", productName, "Please enter a valid name."),
        footer = tagList(
          actionButton("goBackButton", "Go Back")
        )
      ))
      
      # Set showTabs to false
      showTabs <<- FALSE
      
    } else {
      
      # Set showTabs to true
      showTabs <<- FALSE 
      
    }
    
    # Reset loginClicked when viewing in again
    loginClicked(FALSE)
  })
  
  observeEvent(input$goBackButton, {
    removeModal()
    
    # Set showTabs to false
    showTabs <<- FALSE
  })
  
  # Check user name vs input
  output$loggedInUI <- renderUI({
    if (loginClicked()) {
      tagList(
        h3(paste("Welcome,", input$txt1)),
        conditionalPanel(
          condition = "input.loginButton && showTabs",
          tabsetPanel(
            tabPanel("Most Popular Products", tableOutput("table1")),
            tabPanel("Top Selling Products", tableOutput("table2")),
            tabPanel("Products you Frequently Purchased", tableOutput("table3")),
            tabPanel("Products you Purchased the Most", tableOutput("table4")),
            tabPanel("Products you may like", tableOutput("table5"))
          )
        )
      )
    } else {
      tagList(
        conditionalPanel(
          condition = "!input.loginButton || !showTabs",
        tabsetPanel(
            tabPanel("Most Popular Products", tableOutput("table1")),
            tabPanel("Top Selling Products", tableOutput("table2"))  
          )
        )
          )
    }
  })
  
  # Check product name vs input
output$viewedInUI <- renderUI({
  if (viewClicked()) {
    tagList(
      h3(paste(input$txt2, "- Price:", prod_ranking_model$Rate[prod_ranking_model$Product == input$txt2])),
      conditionalPanel(
        condition = "input.viewButton",
        tabsetPanel(
          tabPanel("Customers who purchased this product also purchased these", tableOutput("table6"))
        )
      )
    )
  } else {
    tagList(
      conditionalPanel(
        condition = "!input.viewButton",
        NULL
      )
    )
  }
})
     # Display data in tab 1
  output$table1 <- renderTable({
    most_popular_table <- function() {
      most_popular_prods <- prod_ranking_model[order(prod_ranking_model$Popularity_Rank), c("Product", "Rate")][1:10,]
      rownames(most_popular_prods) <- NULL
      return(most_popular_prods)
    }
    most_popular <- most_popular_table()
    head(most_popular)
  })
  
  # Display data in tab 2
  output$table2 <- renderTable({
    top_sell_table <- function() {
      top_sell_prods <- prod_ranking_model[order(prod_ranking_model$Top_Sell_Rank), c("Product", "Rate")][1:10,]
      rownames(top_sell_prods) <- NULL
      return(top_sell_prods)
    }
    top_sell <- top_sell_table()
    top_sell
  })
  
  # Display data in tab 3
  output$table3 <- renderTable({
    cust_most_popular_table <- function(customerName) {
      cust_most_popular_prods <- subset(cust_prod_ranking_model, Party == customerName)[order(subset(cust_prod_ranking_model, Party == customerName)$Popularity_Rank), c("Product", "Rate")][1:10,]
      rownames(cust_most_popular_prods) <- NULL
      return(cust_most_popular_prods)
    }
    if (loginClicked()) {
      cust_most_popular <- cust_most_popular_table(input$txt1)
      cust_most_popular
    }
  })
  
  # Display data in tab 4
  output$table4 <- renderTable({
    cust_top_sell_table <- function(customerName) {
      cust_top_sell_prods <- subset(cust_prod_ranking_model, Party == customerName)[order(subset(cust_prod_ranking_model, Party == customerName)$Top_Sell_Rank), c("Product", "Rate")][1:10,]
      rownames(cust_top_sell_prods) <- NULL
      return(cust_top_sell_prods)
    }
    if (loginClicked()) {
      cust_top_sell <- cust_top_sell_table(input$txt1)
      cust_top_sell
    }
  })
  
  # Display data in tab 5
  output$table5 <- renderTable({
    recommend_products <- function() {
      similar_custs_corr <- cust_correlation_model[input$txt1, ] %>% sort(desc = TRUE)
      prod_by_similar_custs <- data.frame()
  
      for (i in seq_along(similar_custs_corr)) {
        if (names(similar_custs_corr)[i] != input$txt1) {
          cust_top_sell_prods <- cust_prod_ranking_model[cust_prod_ranking_model$Party == names(similar_custs_corr)[i], ]
          cust_top_sell_prods <- cust_top_sell_prods[, c('Product', 'Qty', 'Rate')]
          cust_top_sell_prods <- cust_top_sell_prods[order(cust_top_sell_prods$Product), ]
          rownames(cust_top_sell_prods) <- NULL         
          corr <- as.numeric(similar_custs_corr[1, i])
          cust_top_sell_prods$Qty_Corr <- cust_top_sell_prods$Qty * corr
          prod_by_similar_custs <- rbind(prod_by_similar_custs, cust_top_sell_prods)
        }
      }
  
      prod_by_similar_custs <- prod_by_similar_custs %>%
        group_by(Product) %>%
        summarise(Qty_Corr = sum(Qty_Corr), Rate = max(Rate))
  
      input_cust_top_sell_prods <- subset(cust_prod_ranking_model, Party == input$txt1)
  
      df_merge <- merge(prod_by_similar_custs, input_cust_top_sell_prods %>% select(Product, No_of_Orders), by = "Product", all.x = TRUE)
  
      prod_recommend_to_cust <- subset(df_merge, is.na(No_of_Orders) | No_of_Orders == "") %>%
        arrange(desc(Qty_Corr)) %>%
        select(Product, Rate) %>%
        head(10) %>%
        ungroup() %>%
        mutate(row_id = row_number()) %>%
        select(-row_id)
  
      return(prod_recommend_to_cust)
    }
    if (loginClicked()) {
      recommend <- recommend_products()
      recommend
    }
  })
  
  # Display data in tab 6
  output$table6 <- renderTable({
    get_similar_products <- function(productName) {
      similar_prods_corr <- prod_correlation_model[input$txt2, ]
      similar_prods_corr <- similar_prods_corr[order(similar_prods_corr, decreasing = TRUE)]
      top_10_products <- names(similar_prods_corr)[1:11]
      top_10_products <- data.frame(Product = top_10_products)
  
      similar_prods <- merge(top_10_products, prod_ranking_model[, c('Product', 'Rate')],
                         by = 'Product', all.x = TRUE)
  
      prod_price <- similar_prods[similar_prods$Product == input$txt2, 'Rate'][1]
      input_prod_index <- which(similar_prods$Product == input$txt2)
      similar_prods <- similar_prods[-input_prod_index, ]
  
      similar_prods <- similar_prods %>%
        mutate(row_id = row_number()) %>%
        select(-row_id)
  
      return(similar_prods)
    }
    if (viewClicked()) {
      similar_products <- get_similar_products(input$txt2)
      similar_products
    }
  })
 observeEvent(input$txt2, {
 outputOptions(output, "table6", suspendWhenHidden = FALSE)
  })  
}
shinyApp(ui = ui, server = server)


Listening on http://127.0.0.1:4985
